In [2]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
from langchain_openai import ChatOpenAI

In [3]:
#taken manually from Raids_Data
#from rows 28, 63, 124, 126, 127, 201, 202, 208, 209, 267, 452, 463.

# the article given in the row 63 doesn't contain the information of arrest_date and end_date
# so, I changed from (2/9/16, 2/23/16) to (None, None)

# test_enddate_labels row124 value changed from "2/22/16" to "2/23/16", after reading the article
# look at the line 'The indictment was returned under seal on Feb. 9, 2016, and unsealed yesterday after the nine defendants were arrested.'
# yesterday refers to Feb 23th

# the article given in the row 126 doesn't contain the information of arrest_date and end_date
# so, I changed from (2/23/16, 2/23/16) to (None, None)

# the article given in the row 127 doesn't contain the information of arrest_date and end_date
# so, I changed from (2/26/16, 2/26/16) to (None, None)

# the article given in the row 201 contains the date information, but the article is Spanish.

# It is hard to find the dates from the article given in the row 202 

# It is hard to find the dates from the article given in the row 208 

# It is hard to find the dates from the article given in the row 267

# It is hard to find the dates from the article given in the row 463


test_arrestdate_labels=["05/18/15", "None","02/09/16","None","None","06/09/16","07/17/16","08/08/16","09/06/16","03/07/17","04/09/18", "04/16/18"]
test_enddate_labels=["06/13/15","None","02/23/16","None","None","06/16/16","07/20/16","08/08/16","09/06/16","03/10/17","04/24/18","04/20/18"]


In [ ]:
api_key = ""

In [21]:
info0=""

with open(directory+"row28_article_text.txt", "r") as row28:
    for line in row28:
        info0 +=line
        
print(info0)

34 Wisconsin men were arrested by Immigration and Customs Enforcement

Posted: Jun 17, 2015 3:04 PM CST

Immigration and Customs Enforcement arrested 280 convicted criminal aliens in six Midwestern states during a month long initiative which ended Saturday.

ICE officers arrested 34 males in Wisconsin during the initiative. All are Mexican nationals, except for one Ecuadorian national who was also arrested. Arrests occurred in the following Wisconsin communities: Arcadia, Deerfield, Franksville, Gillette, Green Bay, Hilbert, Humbird, Janesville, Kenosha, Kewaunee, Madison, Manitowoc, Milwaukee, Princeton, Racine, and Wautoma.

A previously deported 31-year-old Mexican national with a prior burglary conviction was arrested in Madison. He illegally re-entered the United States after being deported in 2012. He remains in ICE custody pending deportation.

In Milwaukee, a previously deported 28-year-old Mexican national with multiple prior convictions for possessing a weapon, possessing coc

In [5]:
prompt_arrestdate0="""Your task is to analyze the given context and determine if the starting date of the U.S. ICE (Immigration and Customs Enforcement) operation is mentioned.

Key details:

The context is a web article about the operation.
The starting date must include the day, month, and year to be considered "mentioned."
The date might be explicitly stated or inferred through time references (e.g., "yesterday," "last week") combined with the article's publication date (if available).

Response instructions:

Respond True if the starting date is mentioned and all three components (day, month, year) can be identified directly or inferred.
Respond False if any component (day, month, year) is missing or cannot be determined.
Do not guess or make up information.
Context: ### {context} ###"""

In [6]:
print(prompt_arrestdate0)

Your task is to analyze the given context and determine if the starting date of the U.S. ICE (Immigration and Customs Enforcement) operation is mentioned.

Key details:

The context is a web article about the operation.
The starting date must include the day, month, and year to be considered "mentioned."
The date might be explicitly stated or inferred through time references (e.g., "yesterday," "last week") combined with the article's publication date (if available).

Response instructions:

Respond True if the starting date is mentioned and all three components (day, month, year) can be identified directly or inferred.
Respond False if any component (day, month, year) is missing or cannot be determined.
Do not guess or make up information.
Context: ### {context} ###


In [6]:
prompt_arrestdate1="""Your task is to analyze the given context and determine the starting date of the U.S. ICE (Immigration and Customs Enforcement) operation.

Key details:

The context is a web article about the operation.
The starting date might be stated explicitly (e.g., a specific date) or implicitly through time references (e.g., "yesterday," "last week").
Use the article's publication date (if available) to infer the starting date based on time references.
Focus only on identifying the operation's starting date. Ignore other dates mentioned in the article.

Response format:

Only provide the date in the format mm/dd/yy (e.g., 04/15/23).
If the starting date cannot be determined, do not guess or infer.
Context: ### {context} ###
"""

In [8]:
print(prompt_arrestdate1)

Your task is to analyze the given context and determine the starting date of the U.S. ICE (Immigration and Customs Enforcement) operation.

Key details:

The context is a web article about the operation.
The starting date might be stated explicitly (e.g., a specific date) or implicitly through time references (e.g., "yesterday," "last week").
Use the article's publication date (if available) to infer the starting date based on time references.
Focus only on identifying the operation's starting date. Ignore other dates mentioned in the article.

Response format:

Only provide the date in the format mm/dd/yy (e.g., 04/15/23).
If the starting date cannot be determined, do not guess or infer.
Context: ### {context} ###



In [7]:
def check_format(response: str)->bool:
    
    print(f"RESPONSE IS {response}")

    if response[0] not in "01":
        return False
    if response[1] not in "0123456789":
        return False
    if response[2]!="/":
        return False
    if response[3] not in "0123":
        return False
    if response[4] not in "0123456789":
        return False    
    if response[5]!="/":
        return False
    if response[6]!="1":
        return False 
    if response[7] not in "45678":
        return False
    else:
        return True
    
    
def model1_arrestdate(model_name: str, info: str, info_name: str, prompt0: str, prompt1: str):
    
    cost=0.0
    
    PROMPT0 = PromptTemplate(template=prompt0, input_variables=["context"])
    
    with get_openai_callback() as cb0:
        llm = LLMChain(
            llm = ChatOpenAI(openai_api_key=api_key,
                     temperature=0.01, model=model_name), prompt=PROMPT0)
    
        response0 = llm.predict(context=info)
    
        cost +=cb0.total_cost
    
    if response0!="True" and response0!="False":
        print("ERROR: Got an unexpected response when looking if the answer is present in the article")
        print(f"UNEXPECTED RESPONSE: {response0}")
        print("The article is:")
        print(info)
        print(f"The article name is {info_name}")
        return None, cost
    
    if response0=="False":
        print("The information you are asking cannot be found only by looking at the given article")
        print("Provide more articles!")
        return None, cost
    
    PROMPT1 = PromptTemplate(template=prompt1, input_variables=["context"])
    
    with get_openai_callback() as cb1:
        llm = LLMChain(
            llm = ChatOpenAI(openai_api_key=api_key,
                     temperature=0.01, model="gpt-4o"), prompt=PROMPT1)
        
        
        response1 = llm.predict(context=info)
    
        cost +=cb1.total_cost
    
    if not (check_format(response1)):
        print("The date is found but is not given in the mm/dd/yy format")
        print(f"Instead got: {response1}")
        return None, cost
    
    print(f"FOUND: {response1}")
    
    
    return response1, cost

            
    

In [8]:
def pipline1_arrestdate(directory, files, model_name):
    
    labels=[]
    cost=0.0
    
    for file in files:
        info=""
        with open(directory+file,"r") as f:
            
            for line in f:
                
                info += line
                
                
        response,c= model1_arrestdate(model_name, info, file, prompt_arrestdate0, prompt_arrestdate1)
        
        cost += c
        
        labels+=["None" if response==None else response]
        
        
    return labels, cost
    
    
    
    
    
    

In [63]:
directory="/Users/canerozer/Desktop/BU/FALL2024/DS701/DS701_Proje/question_answering_experiement/"
test_files=["row28_article_text.txt","row63_article_text.txt","row124_article_text.txt","row126_article_text.txt","row127_article_text.txt","row201_article_text.txt","row202_article_text.txt","row208_article_text.txt","row209_article_text.txt","row267_article_text.txt","row452_article_text.txt","row463_article_text.txt"]

labels, cost= pipline1_arrestdate(directory, test_files, "gpt-4o")

RESPONSE IS 05/18/15
FOUND: 05/18/15
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot be found only by looking at the given article
Provide more articles!
RESPONSE IS 07/17/16
FOUND: 07/17/16
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot be found only by looking at the 

In [64]:
print(labels)

['05/18/15', 'None', 'None', 'None', 'None', 'None', '07/17/16', 'None', 'None', 'None', 'None', 'None']


In [65]:
from sklearn.metrics import accuracy_score

# Metrics
accuracy = accuracy_score(test_arrestdate_labels, labels)

print(f"Accuracy for gpt-4o with model1 is {accuracy}")
print(f"Cost for gpt-4o with model1 is {cost}")

Accuracy for gpt-4o with model1 is 0.4166666666666667
Cost for gpt-4o with model1 is 0.0353525


In [66]:
test_arrestdate_labels

['05/18/15',
 'None',
 '02/09/16',
 'None',
 'None',
 '06/09/16',
 '07/17/16',
 '08/08/16',
 '09/06/16',
 '03/07/17',
 '04/09/18',
 '04/16/18']

In [53]:
labels1, cost1= pipline1_arrestdate(directory, test_files, "gpt-4o-mini")


RESPONSE IS 05/18/15
FOUND: 05/18/15
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot be found only by looking at the given article
Provide more articles!
RESPONSE IS 02/01/16
FOUND: 02/01/16
The information you are asking cannot be found only by looking at the given article
Provide more articles!
RESPONSE IS 05/09/23
The date is found but is not given in the mm/dd/yy format
Instead got: 05/09/23
RESPONSE IS 07/17/16
FOUND: 07/17/16
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot b

In [54]:
labels1

['05/18/15',
 'None',
 'None',
 '02/01/16',
 'None',
 'None',
 '07/17/16',
 'None',
 'None',
 'None',
 'None',
 'None']

In [56]:
accuracy1 = accuracy_score(test_arrestdate_labels, labels1)

print(f"Accuracy for gpt-4o-mini with model1 is {accuracy1}")
print(f"Cost for gpt-4o-mini with model1 is {cost1}")

Accuracy for gpt-4o-mini with model1 is 0.3333333333333333
Cost for gpt-4o-mini with model1 is 0.01048365


In [57]:
labels2, cost2= pipline1_arrestdate(directory, test_files, "gpt-4-turbo")

RESPONSE IS 05/18/15
FOUND: 05/18/15
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot be found only by looking at the given article
Provide more articles!
RESPONSE IS 03/03/16
FOUND: 03/03/16
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot be found only by looking at the given article
Provide more articles!
RESPONSE IS 09/06/16
FOUND: 09/06/16
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking ca

In [58]:
labels2

['05/18/15',
 'None',
 'None',
 'None',
 '03/03/16',
 'None',
 'None',
 'None',
 '09/06/16',
 'None',
 'None',
 'None']

In [59]:
accuracy2 = accuracy_score(test_arrestdate_labels, labels2)

print(f"Accuracy for gpt-4-turbo with model1 is {accuracy2}")
print(f"Cost for gpt-4-turbo with model1 is {cost2}")

Accuracy for gpt-4-turbo with model1 is 0.3333333333333333
Cost for gpt-4-turbo with model1 is 0.13380999999999998


# Running with the Scraped text:

In [9]:
pred_directory="/Users/canerozer/Desktop/BU/FALL2024/DS701/DS701_Proje/text_extraction_experiment/"
pred_files=["pred_row28_article_text.txt","pred_row63_article_text.txt","pred_row124_article_text.txt","pred_row126_article_text.txt","pred_row127_article_text.txt","pred_row201_article_text.txt","pred_row202_article_text.txt","pred_row208_article_text.txt","pred_row209_article_text.txt","pred_row267_article_text.txt","pred_row452_article_text.txt","pred_row463_article_text.txt"]

labels, cost= pipline1_arrestdate(pred_directory, pred_files, "gpt-4o")


/var/folders/_f/tz2yk7vs7yx8cp4kyxh06s2r0000gn/T/ipykernel_74524/1799429007.py:32: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  llm = LLMChain(


RESPONSE IS 05/18/15
FOUND: 05/18/15
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot be found only by looking at the given article
Provide more articles!
RESPONSE IS 07/17/16
FOUND: 07/17/16
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot be found only by looking at the given article
Provide more articles!
The information you are asking cannot be found only by looking at the 

In [10]:
print(labels)

['05/18/15', 'None', 'None', 'None', 'None', 'None', '07/17/16', 'None', 'None', 'None', 'None', 'None']


In [11]:
from sklearn.metrics import accuracy_score

# Metrics
accuracy = accuracy_score(test_arrestdate_labels, labels)

print(f"Accuracy for gpt-4o with model1 is {accuracy}")
print(f"Cost for gpt-4o with model1 is {cost}")

Accuracy for gpt-4o with model1 is 0.4166666666666667
Cost for gpt-4o with model1 is 0.025485
